## Comparing Performance for Pandas, PyArrow and DuckDB

### Performing Common Data Tasks

I want to compare the difference in runtime and storage when using Pandas vs PyArrow vs DuckDB and CSV vs Parquet files.

The data is from Kaggle: https://www.kaggle.com/datasets/geraldooizx/g-coffee-shop-transaction-202307-to-202506

It consists of synthetically generated coffee shop transactions spanning July 2023 to June 2025.

I'll focus on the `transaction` subdirectory which consists of separate CSV file per month, totalling to **1.11 GB**. 

In [21]:
import time
import os
import math
import numpy as np
import pandas as pd
import pyarrow.dataset as ds
import pyarrow as pa
from pathlib import Path
import duckdb

In [22]:
# Get total size of subdirectory.
def get_dir_size(dir_path):
    total_size = 0
    for dirpath, _, filenames in os.walk(dir_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            if not os.path.islink(file_path):
                total_size += os.path.getsize(file_path)
    return convert_size(total_size)

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

dir_size = get_dir_size("./data/transactions")
print(dir_size)

1.11 GB


The following code was used to create the parquet files from the csv files provided.

In [3]:
# Create parquet files with month partitioning:

# csv_dir = Path("data/transactions")
# output_dir = Path("datasets")
# output_dir.mkdir(parents=True, exist_ok=True)
# for csv_file in sorted(csv_dir.glob("*.csv")):
#     month_str = csv_file.stem.split('_')[-1]
#     df = pd.read_csv(csv_file)
#     df['month'] = month_str
#     df.to_parquet(
#         output_dir,
#         engine="pyarrow",
#         partition_cols=["month"],
#         index=False
#     )
#     print(f"Processed {csv_file.name} with partition month={month_str}")

In [23]:
dir_size = get_dir_size("./datasets")
print(dir_size)

701.39 MB


Storing the same transaction data in Parquet files, partitioned by month, the resulting data is of size **701.39 MB**.

In [24]:
tic = time.perf_counter()
df1 = pd.read_csv("data/transactions/transactions_202307.csv")
df2 = pd.read_csv("data/transactions/transactions_202308.csv")
df3 = pd.read_csv("data/transactions/transactions_202309.csv")
toc = time.perf_counter()
print(f"Loaded dataset in {toc - tic:0.4f} seconds")
df1["month"] = "202307"
df2["month"] = "202308"
df3["month"] = "202309"
df_from_pd = pd.concat([df1, df2, df3])
print(df_from_pd.head())

Loaded dataset in 3.3465 seconds
                         transaction_id  store_id  payment_method_id  \
0  2ae6d188-76c2-4095-b861-ab97d3cd9312         4                  5   
1  7d0a474d-62f4-442a-96b6-a5df2bda8832         7                  1   
2  85f86fef-fddb-4eef-9dc3-1444553e6108         1                  5   
3  4c41d179-f809-4d5a-a5d7-acb25ae1fe98         5                  2   
4  51e44c8e-4812-4a15-a9f9-9a46b62424d6         8                  5   

   voucher_id  user_id  original_amount  discount_applied  final_amount  \
0         NaN      NaN             38.0               0.0          38.0   
1         NaN      NaN             33.0               0.0          33.0   
2         NaN      NaN             27.0               0.0          27.0   
3         NaN      NaN             45.5               0.0          45.5   
4         NaN      NaN             27.0               0.0          27.0   

            created_at   month  
0  2023-07-01 07:00:00  202307  
1  2023-07-01 07:

In [25]:
tic = time.perf_counter()
dataset = ds.dataset("datasets", format="parquet", partitioning="hive")
filter_months = ds.field("month").isin(["202307", "202308", "202309"])
table = dataset.to_table(filter=filter_months)
toc = time.perf_counter()
print(f"Loaded dataset in {toc - tic:0.4f} seconds")
top_row = table.slice(0, 1)
print(top_row)

Loaded dataset in 0.4053 seconds
pyarrow.Table
transaction_id: string
store_id: int64
payment_method_id: int64
voucher_id: double
user_id: double
original_amount: double
discount_applied: double
final_amount: double
created_at: string
month: int32
----
transaction_id: [["2ae6d188-76c2-4095-b861-ab97d3cd9312"]]
store_id: [[4]]
payment_method_id: [[5]]
voucher_id: [[null]]
user_id: [[null]]
original_amount: [[38]]
discount_applied: [[0]]
final_amount: [[38]]
created_at: [["2023-07-01 07:00:00"]]
month: [[202307]]


One run sample found that loading in the first three months of data from:
- CSV files with Pandas took **3.2922** seconds
- Parquet files with PyArrow took **0.4542** seconds

**I've found PyArrow to be roughly 7 times quicker loading in data with Parquet files than Pandas is loading in data from CSV.**

#### Sorting Columns

In [26]:
tic = time.perf_counter()
sorted_df = df_from_pd.sort_values('final_amount', ascending=False)
toc = time.perf_counter()
print(f"Sorted by column in {toc - tic:0.4f} seconds")
print(sorted_df.head())

Sorted by column in 1.6015 seconds
                              transaction_id  store_id  payment_method_id  \
540192  d04def13-2c6a-49de-ab00-829d5c9742b6         8                  5   
459259  f506afa3-d30c-42d6-944d-3f5f83080215         2                  5   
187612  0e569659-fb7b-4c89-8233-9434ca9d4a28         3                  1   
461209  ab1e6405-b734-4082-8a43-c9c6e636942b         7                  4   
341074  42cb9509-30f2-454f-adef-a29a83da34ab         4                  4   

        voucher_id  user_id  original_amount  discount_applied  final_amount  \
540192         NaN      NaN             90.0               0.0          90.0   
459259         NaN      NaN             90.0               0.0          90.0   
187612         NaN      NaN             90.0               0.0          90.0   
461209         NaN      NaN             90.0               0.0          90.0   
341074         NaN      NaN             90.0               0.0          90.0   

                 crea

In [27]:
tic = time.perf_counter()
sorted_table = table.sort_by([("final_amount", "descending")])
toc = time.perf_counter()
print(f"Sorted by column in {toc - tic:0.4f} seconds")
top_row = sorted_table.slice(0, 1)
print(top_row)

Sorted by column in 1.0852 seconds
pyarrow.Table
transaction_id: string
store_id: int64
payment_method_id: int64
voucher_id: double
user_id: double
original_amount: double
discount_applied: double
final_amount: double
created_at: string
month: int32
----
transaction_id: [["2346bcfd-520f-43ba-9599-43934670dfc3"]]
store_id: [[2]]
payment_method_id: [[5]]
voucher_id: [[null]]
user_id: [[null]]
original_amount: [[90]]
discount_applied: [[0]]
final_amount: [[90]]
created_at: [["2023-07-01 13:49:23"]]
month: [[202307]]


#### Filtering

In [28]:
tic = time.perf_counter()
month7_no_voucher = df_from_pd[(df_from_pd["month"] == "202307") & (df_from_pd["discount_applied"] == 0)]
toc = time.perf_counter()
print(f"Filtered in {toc - tic:0.4f} seconds")
print(month7_no_voucher.head())

Filtered in 0.1806 seconds
                         transaction_id  store_id  payment_method_id  \
0  2ae6d188-76c2-4095-b861-ab97d3cd9312         4                  5   
1  7d0a474d-62f4-442a-96b6-a5df2bda8832         7                  1   
2  85f86fef-fddb-4eef-9dc3-1444553e6108         1                  5   
3  4c41d179-f809-4d5a-a5d7-acb25ae1fe98         5                  2   
4  51e44c8e-4812-4a15-a9f9-9a46b62424d6         8                  5   

   voucher_id  user_id  original_amount  discount_applied  final_amount  \
0         NaN      NaN             38.0               0.0          38.0   
1         NaN      NaN             33.0               0.0          33.0   
2         NaN      NaN             27.0               0.0          27.0   
3         NaN      NaN             45.5               0.0          45.5   
4         NaN      NaN             27.0               0.0          27.0   

            created_at   month  
0  2023-07-01 07:00:00  202307  
1  2023-07-01 07:00:02 

In [29]:
tic = time.perf_counter()
filtered_table = table.filter(pa.compute.equal(table['month'], 202307))
filtered_table = filtered_table.filter(pa.compute.equal(filtered_table['discount_applied'], 0))
toc = time.perf_counter()
print(f"Sorted by column in {toc - tic:0.4f} seconds")
top_row = filtered_table.slice(0, 1)
print(top_row)

Sorted by column in 0.1234 seconds
pyarrow.Table
transaction_id: string
store_id: int64
payment_method_id: int64
voucher_id: double
user_id: double
original_amount: double
discount_applied: double
final_amount: double
created_at: string
month: int32
----
transaction_id: [["2ae6d188-76c2-4095-b861-ab97d3cd9312"]]
store_id: [[4]]
payment_method_id: [[5]]
voucher_id: [[null]]
user_id: [[null]]
original_amount: [[38]]
discount_applied: [[0]]
final_amount: [[38]]
created_at: [["2023-07-01 07:00:00"]]
month: [[202307]]


#### Grouping and Aggregating

In [30]:
tic = time.perf_counter()
summary = df_from_pd.groupby(["month"]).agg({'final_amount': ['min', 'max', lambda x: np.mean(x)]})
toc = time.perf_counter()
print(f"Aggregated in {toc - tic:0.4f} seconds")
print(summary)

Aggregated in 0.2226 seconds
       final_amount                 
                min   max <lambda_0>
month                               
202307         5.58  90.0  33.120199
202308         1.00  90.0  32.933415
202309         1.00  90.0  32.774340


In [31]:
tic = time.perf_counter()
summary = table.group_by("month").aggregate([
   ("final_amount", "min"),
   ("final_amount", "max"),
   ("final_amount", "mean")
])
toc = time.perf_counter()
print(f"Aggregated in {toc - tic:0.4f} seconds")
print(summary)

Aggregated in 0.0382 seconds
pyarrow.Table
month: int32
final_amount_min: double
final_amount_max: double
final_amount_mean: double
----
month: [[202307,202308,202309]]
final_amount_min: [[5.58,1,1]]
final_amount_max: [[90,90,90]]
final_amount_mean: [[33.12019873859749,32.93341456176001,32.77434009886071]]


As well as loading data in, performing common data tasks is much faster using PyArrow compared to Pandas.

#### Merging

In [32]:
# Create dataframe to merge
locations = {
	'store_id': [
		1, 2, 3, 4, 5, 6, 7, 8, 9, 10
	], 
	'location': [
		'London',
		'Manchester',
		'Liverpool',
		'Newcastle',
		'Glasgow',
		'Norwich',
		'Bath',
		'Bristol',
		'Nottingham',
		'Coventry'
	]
}
stores_df = pd.DataFrame(locations)
tic = time.perf_counter()
merged_data = df_from_pd.merge(stores_df)
toc = time.perf_counter()
print(f"Merged in {toc - tic:0.4f} seconds")
print(merged_data.head())

Merged in 0.7031 seconds
                         transaction_id  store_id  payment_method_id  \
0  2ae6d188-76c2-4095-b861-ab97d3cd9312         4                  5   
1  7d0a474d-62f4-442a-96b6-a5df2bda8832         7                  1   
2  85f86fef-fddb-4eef-9dc3-1444553e6108         1                  5   
3  4c41d179-f809-4d5a-a5d7-acb25ae1fe98         5                  2   
4  51e44c8e-4812-4a15-a9f9-9a46b62424d6         8                  5   

   voucher_id  user_id  original_amount  discount_applied  final_amount  \
0         NaN      NaN             38.0               0.0          38.0   
1         NaN      NaN             33.0               0.0          33.0   
2         NaN      NaN             27.0               0.0          27.0   
3         NaN      NaN             45.5               0.0          45.5   
4         NaN      NaN             27.0               0.0          27.0   

            created_at   month   location  
0  2023-07-01 07:00:00  202307  Newcastle  
1  

In [33]:
# PyArrow does not have direct join functionality.
# Time loading in and converting to pandas.
tic = time.perf_counter()
dataset = ds.dataset("datasets", format="parquet", partitioning="hive")
filter_months = ds.field("month").isin(["202307", "202308", "202309"])
table = dataset.to_table(filter=filter_months)
df_from_arrow = table.to_pandas()
toc = time.perf_counter()
print(f"Created table as pandas DataFrame in {toc - tic:0.4f} seconds")

Created table as pandas DataFrame in 2.0802 seconds


**PyArrow does not currently support join/merge functionality**. Instead, we could convert the table to a Pandas DataFrame. Loading the data from Parquet files and converting to a dataframe is **faster** than loading the data from CSV directly to a Pandas DataFrame.

Alternatively, we can use **DuckDB**!

### DuckDB

[DuckDB](https://duckdb.org/) is a **"fast open-source database system"**. It has a plethora of capabilities; here, I'll be using an **in-memory** database, where no data is persisted to disk and all data is lost when a process finishes.

DuckDB runs embedded inside our application (Python, R, Java, C++, etc.). It uses a columnar data layout and reads Parquet without loading the data into memory. There is **no server required**.

Below are two examples: 
- Performing a join on two parquet files.
- Performing aggregation on a csv file without loading the data into memory.

#### Performing a Join

The following code is used to save the `store_df` dataframe from earlier to Parquet.

In [ ]:
# locations = {
# 	'store_id': [
# 		1, 2, 3, 4, 5, 6, 7, 8, 9, 10
# 	], 
# 	'location': [
# 		'London',
# 		'Manchester',
# 		'Liverpool',
# 		'Newcastle',
# 		'Glasgow',
# 		'Norwich',
# 		'Bath',
# 		'Bristol',
# 		'Nottingham',
# 		'Coventry'
# 	]
# }
# store_df = pd.DataFrame(locations)
# store_df.to_parquet("store_df.parquet", engine="pyarrow")

We connect to a local in-process DuckDB database (no server needed), then **execute SQL** on the desired parquet files.

In [34]:
con = duckdb.connect()
transactions07 = "datasets/month=202307"
locations = "datasets/store_df.parquet"
tic = time.perf_counter()
df = con.execute("""
    SELECT
        t1.transaction_id,
        t1.store_id,
		t1.original_amount,
		t2.location
    FROM read_parquet(?) AS t1
    JOIN read_parquet(?) AS t2
        USING (store_id)
    WHERE t1.original_amount > 30
""", [transactions07, locations]).df()
toc = time.perf_counter()
print(f"DuckDB merge executed in {toc - tic:0.4f} seconds")
print(df)

DuckDB merge executed in 0.3978 seconds
                              transaction_id  store_id  original_amount  \
0       2ae6d188-76c2-4095-b861-ab97d3cd9312         4             38.0   
1       7d0a474d-62f4-442a-96b6-a5df2bda8832         7             33.0   
2       4c41d179-f809-4d5a-a5d7-acb25ae1fe98         5             45.5   
3       d449cf8f-e6d5-4b09-a02e-693c7889dee8         8             45.0   
4       6b00c575-ec6e-4070-82d2-26d66b017b8b         3             77.0   
...                                      ...       ...              ...   
314897  e9915869-eabf-4fa8-9a48-a506f3313ecb         4             54.0   
314898  40576233-9726-4fcc-bcb6-a623ad9f30e8         8             48.0   
314899  ad2fe05e-7581-4d7b-8ec6-6d94ca29acfd         5             33.0   
314900  9d6c44ee-e753-4527-a6ef-e2f1cc1c8c99         8             54.0   
314901  161dfdb3-08df-411c-9221-12a7c2448066         2             64.0   

          location  
0        Newcastle  
1             Bat

In one run, this took **0.2561** seconds!

#### Aggregation with a CSV File

We can also perform operations on data in a csv file without reading the data into memory.

In [35]:
con = duckdb.connect()
tic = time.perf_counter()
result = con.execute("""
    SELECT
		store_id,
        COUNT(*) AS n_transactions,
        AVG(original_amount) AS avg_orig,
        AVG(final_amount) AS avg_final
    FROM read_csv_auto('data/transactions/transactions_202307.csv')
    GROUP BY store_id
    ORDER BY store_id
""").df()
toc = time.perf_counter()
print(f"DuckDB merge executed in {toc - tic:0.4f} seconds")
print(result)


DuckDB merge executed in 0.3518 seconds
   store_id  n_transactions   avg_orig  avg_final
0         1           62512  33.252536  33.091565
1         2           61664  33.295999  33.140107
2         3           61904  33.271566  33.113469
3         4           61960  33.301404  33.142113
4         5           61970  33.247475  33.092544
5         6           62186  33.288513  33.136463
6         7           62004  33.230695  33.074163
7         8           62338  33.243110  33.080278
8         9           61742  33.397468  33.236847
9        10           61982  33.257252  33.095423


In one run this took **0.2613** seconds!

### Summary

It's safe to say that DuckDB is incredibly powerful, from its high speed operations to flexibility and low over-head. If you are already familiar with SQL, this is a nice, clean and fast approach.

Despite the current limitation with performing joins in PyArrow, it is definitly worth using this when working with large amounts of data in memory. There is a clear speed performance benefit compared to Pandas, and for those familar with Pandas, there isn't a big difference in syntax.

Finally, Parquet files are great way of storing lots of data. I found my local machine struggling to even store the csv example data in memory, but once I created the Parquet files this was not an issue. Partitioning the data is especially useful. This is a powerful data file format for storing and reading data!